In [52]:
import os
import json
import pandas as pd 
import traceback

In [53]:
from langchain.chat_models import ChatOpenAI

from  dotenv import load_dotenv


In [54]:
load_dotenv()
llm = ChatOpenAI(openai_api_key=os.getenv('KEY'), model_name="gtp-3.5-turbo", temperature =0.5)

In [55]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [56]:
RESPONSE_JSON ={
    "1":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here ",
            "b":"choice here ",
            "c":"choice here ",
            "d":"choice here ",
                 },
        "correct":"correct answer",
            },
    "2":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here ",
            "b":"choice here ",
            "c":"choice here ",
            "d":"choice here ",
                 },
        "correct":"correct answer",
            },
    "3":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here ",
            "b":"choice here ",
            "c":"choice here ",
            "d":"choice here ",
                 },
        "correct":"correct answer",
            },
    "4":{
        "mcq":"multiple choice question",
        "option":{
            "a":"choice here ",
            "b":"choice here ",
            "c":"choice here ",
            "d":"choice here ",
                 },
        "correct":"correct answer",
            },

}

In [57]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your jobe to \
create a quiz of {numbers} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to fornate your respones like RESPONSE_JSON below and use it a guide.
Ensure to make {numbers} MCQs
### RESPONSE_JSON
{response_json}
    
"""

In [58]:


quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "numbers", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [59]:
quize_chain= LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [60]:
TEMPLATE2 ="""
    You are an expert english grammarian and writer. given a Multiple choice Quiz fro {subject} students.\
    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity 
    if the quiz is not at per with the conngitive and analytical abilities of the students,\
    update the quiz questions which needs to be changes and change the tone such that it perfectly fits the students .
    Quiz_MCQs:
    {quiz}

    Check from an export English writer of the above quize:
"""

In [61]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"], template=TEMPLATE2
)

In [62]:
review_Chian = LLMChain(llm=llm, prompt=quiz_evaluation_prompt , output_key="review", verbose=True)

In [65]:
BOOT = SequentialChain(chains=[quize_chain, review_Chian], input_variables=["text", "numbers", "subject", "tone", "response_json"],
                       output_variables=["quiz", "review"],verbose=True)

In [68]:
fil_path="../data.txt"
with open (fil_path, "r") as file:
    text = file.read()

print(text)

As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]:â€ٹ488â€ٹ

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]:â€ٹ488â€ٹ By 1980, expert systems had come to dominate AI, and statistics was out of favor.[24] Work on symbolic/knowledge-based learning did continue within AI, leading to inductiv

In [69]:
#Serializig the python dictionary into a JSON-formate string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "option": {"a": "choice here ", "b": "choice here ", "c": "choice here ", "d": "choice here "}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "option": {"a": "choice here ", "b": "choice here ", "c": "choice here ", "d": "choice here "}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "option": {"a": "choice here ", "b": "choice here ", "c": "choice here ", "d": "choice here "}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "option": {"a": "choice here ", "b": "choice here ", "c": "choice here ", "d": "choice here "}, "correct": "correct answer"}}'

In [70]:
Number=5
subject="MACHINE LEARNING"
tone="simple"
with get_openai_callback() as cb:
    response=BOOT(
        {
            "text":text,
            "numbers":Number,
            "subject":subject,
            "tone":tone,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\nane\AppData\Local\Temp\ipykernel_18604\2942641268.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=BOOT(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[22] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[23]:â€ٹ488â€ٹ

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[23]:â€ٹ488â€ٹ By 1980, expert systems had come to dominate AI, and statistics

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gtp-3.5-turbo` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
quiz=response.get("quiz")

In [ ]:

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:

quiz_table_data

In [ ]:

quiz=pd.DataFrame(quiz_table_data)

In [81]:
quiz.to_csv("machinelearning.csv",index=False)
from datetime import datetime


NameError: name 'quiz' is not defined

In [83]:

from datetime import datetime 

datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'09_22_2024_03_25_39'